### Лабораторная работа № 3. Вариант №4
##### Автор: Торопкин Р.А.
##### Проверил: к.т.н Балашева И.Ю. 

In [35]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

Приведение задачи к каноническому виду и формирование симплекс таблицы

In [36]:
def ctc(A_u, b_u, c, sign):
    size = np.shape(A_u)
    sol_index = []
    c_length = len(c)
    dop_basis_length = 0
    x_solution = [0 for i in range(0, c_length)]
    
    for s in sign:
        if s != '=':
            dop_basis_length+=1
           
    if dop_basis_length != 0:
        new_A = np.concatenate((A_u, np.eye(dop_basis_length)), axis=1)
        for i in range(0, len(sign)):
            if sign[i] == '>=':
                new_A[i] = np.dot(new_A[i], -1)
                new_A[i][len(new_A[i]) - size[0] + i] = 1
                b_u[i] = -b_u[i]
        c = np.concatenate((-c, np.zeros((1,dop_basis_length))[0]))
    else:
        new_A = A_u
        c = -c
    
    b_u = np.concatenate((b_u, [0])).reshape((len(b_u)+1,1))
    index_arr = []
    columns_arr = [] 
    for i in range(0, size[0]):
        index_arr.append(f'x{len(new_A[0]) - size[0] + i + 1}')
    index_arr.append('Z')
    for i in range(0, len(new_A[0])):
        columns_arr.append(f'x{i+1}')
        if i < c_length:
            sol_index.append(f'x{i+1}')
    columns_arr.append('b')
    table = np.vstack((new_A, c))
    table = np.hstack((table, b_u))
    df = pd.DataFrame(data=table, index=index_arr, columns=columns_arr)
    sol_index.append('Z')
    return df, index_arr, columns_arr, sol_index, x_solution
    

Выбор способа решения задачи.

True - двойственный метод

False - обычный

In [37]:
def checkMethod(table):
    lastColumn = table.shape[1] - 1
    if min(table[:-1,lastColumn]) < 0:
        return True
    else:
        return False

Проверка разрешимости задачи для двойственного метода

In [38]:
def checkSolution(table):
    lastColumn = table.shape[1] - 1
    b = table[:,lastColumn]
    for i in range(0, len(b)):
        if b[i] < 0:
            neg_count = len(list(filter(lambda x: (x < 0), table[i,:lastColumn])))
            if neg_count == 0:
                return False
    return True

Выбор разрешающей строки

In [39]:
def getSolRow(table):
    lastColumn = table.shape[1] - 1
    b = table[:,lastColumn]
    index = 0
    val = 0
    for i in range(0, len(b)):
        if b[i] < 0:
            if abs(b[i]) > val:
                val = abs(b[i])
                index = i
    return index

Вычисление двойственных отношений и определение разрешающего столбца p

In [40]:
def getSolCol(q, table):
    z_index = table.shape[0] - 1
    length = table.shape[1] - 1
    d = [np.inf for i in range(0, length)]
    for j in range(0, length):
        if table[q,j] < 0:
            d[j] = abs(table[z_index,j] / table[q,j])
    print('Двойственные отношения:',d)
    return np.flatnonzero(d==np.min(d))[0]

Проверка разрешимости задачи для обычного метода на максимум

In [41]:
def checkDefaultMaxSolution(table, p):
    count = 0
    size = table.shape
    for i in range(0, size[0]):
        if table[i][p] <= 0:
            count+=1
    if count == size[0]:
        return False
    else:
        return True

Проверка разрешимости задачи для обычного метода на минимум

In [42]:
def checkDefaultMinSolution(table, p):
    count = 0
    size = table.shape
    for i in range(0, size[0]):
        if table[i][p] >= 0:
            count+=1
    if count == size[0]:
        return False
    else:
        return True

Вычесление разрешающего стобца для обычного метода на максимум

In [43]:
def getDefaultMaxCol(row):
    mini = 0
    min_index = -1
    for i in range(0, len(row)):
        if row[i] < mini:
            min_index = i
            mini = row[i]
    return min_index

Вычесление разрешающего стобца для обычного метода на минимум

In [44]:
def getDefaultMinCol(row):
    maxi = - np.inf
    max_index = -1
    for i in range(0, len(row)):
        if row[i] > maxi:
            max_index = i
            maxi = row[i]
    return max_index

Вычесление разрешающей строки для обычного метода

In [45]:
def getDefaultRow(table, p):
    mini = np.inf
    min_index = -1
    size = table.shape
    for i in range(0, size[0]-1):
        if table[i][p] > 0:
            s = table[i][size[1] - 1] / table[i][p]
            if s < mini:
                mini = s
                min_index = i
    return min_index

Пересчет таблицы по правилам обычного симплекс метода

In [46]:
def calculate(table, q,p):
    qp = table[q][p]
    size = table.shape
    for j in range(0, size[1] - 1):
        table[q][j] = table[q][j] / qp
    table[q][size[1] - 1] = table[q][size[1] - 1] / qp
    for i in range(0, size[0]):
        if i != q:
            ip = -table[i][p]
            for j in range(0, size[1] - 2):
                table[i][j] = table[i][j] + table[q][j] * ip
            table[i][size[1] - 1] = table[i][size[1] - 1] + table[q][size[1] - 1] * ip

Определение нового базиса

In [47]:
def newBasis(q,p, table, index_arr, columns_arr):
    index_arr[q] = columns_arr[p]
    return pd.DataFrame(data=table, index=index_arr, columns=columns_arr)
    

Вычисление максимального значения функции

Исходные данные задачи. 

![Исходные данные](https://i.ibb.co/dc9c4kr/image.png)

In [48]:
A_u = np.array([[2,1,-2],
                [-1,0,1],
                [-2,1,2]]) # Система ограничений
b_u = np.array([1,1,1]) # Свободные члены
c = np.array([1,1,1]) # Коэффициенты целевой функции
sign = ['>=','>=','>='] # Знаки неравенства ограничений
isMin = True
table, index_arr, columns_arr, sol_index, x_solution = ctc(A_u, b_u, c, sign) #Симплексная таблица
iters = 0
size = table.shape # Размер таблицы
print('Исходная симплексная таблица')
table

Исходная симплексная таблица


,x1,x2,x3,x4,x5,x6,b
x4,-2.0,-1.0,2.0,1.0,0.0,0.0,-1.0
x5,1.0,0.0,-1.0,0.0,1.0,0.0,-1.0
x6,2.0,-1.0,-2.0,0.0,0.0,1.0,-1.0
Z,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0


In [49]:
isSolve = False
print(table)
while True:
    iters+=1
    print('Итерация',iters)
    if checkMethod(table.values):
        if checkSolution(table.values):
            q = getSolRow(table.values)
            p = getSolCol(q, table.values)
        else:
            print('Задача неразрешима в силу несовместности системы ограничений, расчет прекращается')
            break
    else:
        if isMin:
            p = getDefaultMinCol(table.values[size[0]-1])
        else:
            p = getDefaultMaxCol(table.values[size[0]-1])
        if p == -1:
            print('Найдено решение')
            isSolve = True
            break
        print(p)
        if isMin:
            if checkDefaultMinSolution(table.values, p):
                q = getDefaultRow(table.values, p)
            else:
                print('Оптимальное решение отсутствует, вычисления прекращаются.')
                break
        else:
            if checkDefaultMaxSolution(table.values, p):
                q = getDefaultRow(table.values, p)
            else:
                print('Оптимальное решение отсутствует, вычисления прекращаются.')
                break
    calculate(table.values,q,p)
    print('Новая таблица')
    table = newBasis(q,p,table.values, index_arr,columns_arr)
    print(table)
if isSolve:
    for i in range(0, len(c)):
        col = 'x' + str(i+1)
        if col in table.index.values:
            x_solution[i] = table['b'][col]
    x_solution.append(table['b']['Z'])
    print('Итоговое решение')
    print(pd.DataFrame(data=[x_solution], columns=sol_index))

     x1   x2   x3   x4   x5   x6    b
x4 -2.0 -1.0  2.0  1.0  0.0  0.0 -1.0
x5  1.0  0.0 -1.0  0.0  1.0  0.0 -1.0
x6  2.0 -1.0 -2.0  0.0  0.0  1.0 -1.0
Z  -1.0 -1.0 -1.0  0.0  0.0  0.0  0.0
Итерация 1
Двойственные отношения: [0.5, 1.0, inf, inf, inf, inf]
Новая таблица
     x1   x2   x3   x4   x5   x6    b
x1  1.0  0.5 -1.0 -0.5 -0.0 -0.0  0.5
x5  0.0 -0.5  0.0  0.5  1.0  0.0 -1.5
x6  0.0 -2.0  0.0  1.0  0.0  1.0 -2.0
Z   0.0 -0.5 -2.0 -0.5  0.0  0.0  0.5
Итерация 2
Двойственные отношения: [inf, 0.25, inf, inf, inf, inf]
Новая таблица
     x1   x2   x3    x4   x5   x6    b
x1  1.0  0.0 -1.0 -0.25  0.0 -0.0  0.0
x5  0.0  0.0  0.0  0.25  1.0  0.0 -1.0
x2 -0.0  1.0 -0.0 -0.50 -0.0 -0.5  1.0
Z   0.0  0.0 -2.0 -0.75  0.0  0.0  1.0
Итерация 3
Задача неразрешима в силу несовместности системы ограничений, расчет прекращается


Проверка решения с помощью встроенной функции. Приводим коэффициенты ограничений в соответствие со знаком

In [50]:
# A_u[1] = -A_u[1]
# inner_solution = linprog(-c, A_u, b_u, method='simplex')
# print(f'Минимальное значение x: {inner_solution.x}\nОптимальное решение: {-inner_solution.fun}')
A_u = np.array([[-2,-1,2],
                [1,0,-1],
                [2,-1,-2]])
b_u = np.array([1,1,1])
c = np.array([1,1,1])
inner_solution = linprog(c, A_u, b_u, method='simplex')
print(f'Минимальное значение x: {inner_solution.x}\nОптимальное решение: {-inner_solution.fun}')

Минимальное значение x: [0. 0. 0.]
Оптимальное решение: -0.0


### Контрольные вопросы:

#### 1. Можно ли для задачи линейного программирования, содержащей в системе ограничений неравенства разных направлений, построить двойственную задачу?

Можно, для этого необходимо все неравенства системы привести к одному направлению: >= в задаче на минимум или <= в задаче на максимум.

#### 2. Если в основной задаче отсутствуют условия неотрицательности переменных, то какие последствия это влечет в двойственной задаче?

Если в исходной задаче на некоторую переменную Xj не наложено условие неотрицательности, то соответствующее ей j-е ограничение двойственной задачи является равенством

![Исходные данные](https://i.ibb.co/19kV9vH/image.png)

#### 3. Чем отличаются матрицы систем ограничений в паре двойственных задач?

Матрица системы ограничений двойственной задачи получается в результате транспонирования исходной системы, в качестве свободных членов выступают коэффициенты целевой функции

#### 4. Какова связь между экстремальными значениями пары двойственных задач линейного программирования?

Целевая функция в одной задаче задается на max, в другой – на min.

#### 5. Что можно сказать о решении двойственной задачи, если решение основной задачи не существует по причине несовместимости ее системы ограничений?

Так как понятие двойственности является взаимным, то двойственная задача является и основной. 
Первая теорема двойственности гласит, что если одна из двойственных задач не разрешима, то и другая тоже не имеет решений.

#### 6. Можно ли производить симплексное преобразование, используя отрицательный разрешающий коэффициент?

Только он и будет учавствовать в сиплексном преобразовании, исходя из условия разрешимости задачи

#### 7. Могут ли обе двойственные задачи быть задачами на максимум?

Нет, так как это уже будет другая задача и из одной задачи I1 нельзя будет получить другую задачу I2.

Если одна из двойственных задач имеет оптимальное решение, то другая также имеет оптимальное решение, причем для экстремальных значений целевых функций выполняется соотношение Zmax = Tmin